In [1]:
# Run these following 3 installs to if you don't have shapely, geopands, etc
# if you are using sagemaker, you should use the "conda_python3" kernel
#!pip install laspy
#!pip install geopandas
#!pip install shapely

In [2]:
# Run these following 2 installs to install/upgrade database interface modules
#!pip install sqlalchemy psycopg2
#!pip install psycopg2-binary
!pip install geoalchemy2

You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
### install/upgrade folium mapping tools
# !pip install folium

In [4]:
import boto3
import json

In [5]:
import numpy as np
import pandas as pd
import psycopg2
import sqlalchemy as db
import matplotlib as plt

%matplotlib inline

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [6]:
from sqlalchemy import update 
from geoalchemy2 import Geometry
from sqlalchemy.dialects import postgresql

In [7]:
#Get information from Secrets Manager, especially about the database
secretsmanager_client = boto3.client('secretsmanager')
dbSecretValues = secretsmanager_client.get_secret_value(SecretId='Postgres-GISDB')

In [8]:
dbSecretValuesJson = json.loads(dbSecretValues['SecretString'])
#uncomment the following line to confirm the Database access information
#dbSecretValuesJson 

In [9]:
from sqlalchemy import create_engine
# Postgres username, password, and database name
POSTGRES_ADDRESS = dbSecretValuesJson['POSTGRES_ADDRESS'] ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = int(dbSecretValuesJson['POSTGRES_PORT'])
POSTGRES_USERNAME = dbSecretValuesJson['POSTGRES_USERNAME'] ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = dbSecretValuesJson['POSTGRES_PASSWORD'] ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD POSTGRES_DBNAME = 'database' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_DBNAME = dbSecretValuesJson['POSTGRES_DBNAME']
# A long string that contains the necessary Postgres login information
# postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
#     username=POSTGRES_USERNAME,
#     password=POSTGRES_PASSWORD,
#     ipaddress=POSTGRES_ADDRESS,
#     port=POSTGRES_PORT,
#     dbname=POSTGRES_DBNAME))
postgres_str = f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}/{POSTGRES_DBNAME}"

engine = create_engine(postgres_str)
connection = engine.connect()
metadata = db.MetaData()
building_detailed_info_table = db.Table('building_detailed_info_within_ajjits_selected_tracts', metadata, autoload=True, autoload_with=engine)

In [10]:
#Get the las grid that belong to a building, get the percentil numbers and enter them into the table
#engie is the postgres sqlalchemy engine
def query_and_update_sql_table_with_height_pct(bldgid, engine, connection, building_detailed_info_table): 
#     sql_string = "select * from building_detailed_info_within_ajjits_selected_tracts where bldgid = 59026".format(bldgid)
#     pd.read_sql_query(sql_string, engine)
    sql_string = """select las_grnd_coord, lidar_z, intensity, classification, scan_angle, synthetic, withheld from lidarbuildingmatchtest 
        where bldgid = {0}""".format(bldgid)
    las_within_bldg_border_df = pd.read_sql_query(sql_string, engine)
    las_qt_results_df = las_within_bldg_border_df[['lidar_z']].quantile([.999, .995, .99, .95, .90])
    pct999 = las_qt_results_df.loc[.999, 'lidar_z']
    pct995 = las_qt_results_df.loc[.995, 'lidar_z']
    pct990 = las_qt_results_df.loc[.990, 'lidar_z']
    pct950 = las_qt_results_df.loc[.950, 'lidar_z']
    pct900 = las_qt_results_df.loc[.900, 'lidar_z']

    query = db.update(building_detailed_info_table).values({'pct999': pct999, 'pct995': pct995, 'pct990': pct990, 'pct950': pct950})
    query = query.where(building_detailed_info_table.columns.bldgid == bldgid)
    ResultProxy = connection.execute(query)
    ResultProxy

In [11]:
query_and_update_sql_table_with_height_pct(59450, engine, connection, building_detailed_info_table)

In [12]:
sql_string = """select bldgid from building_detailed_info_within_ajjits_selected_tracts"""
bldgid_within_ajjit_tracts = pd.read_sql_query(sql_string, engine)
bldgid_within_ajjit_tracts.head()

,bldgid
0,59547
1,59576
2,59727
3,59730
4,59769


In [14]:
for index, row in bldgid_within_ajjit_tracts.iterrows():
    print (index, row['bldgid'])
    bldgid = np.int(row['bldgid'])
    query_and_update_sql_table_with_height_pct(bldgid, engine, connection, building_detailed_info_table)

0 59547
1 59576
2 59727
3 59730
4 59769
5 59889
6 59930
7 59909
8 59945
9 60188
10 60220
11 60285
12 60289
13 60416
14 60421
15 60512
16 60517
17 60526
18 60527
19 60528
20 60558
21 60634
22 60641
23 60660
24 60769
25 60771
26 60806
27 60855
28 61016
29 61069
30 61095
31 61556
32 61584
33 61715
34 61747
35 61982
36 62036
37 62042
38 62135
39 62173
40 62174
41 62232
42 62283
43 62324
44 62378
45 62457
46 62538
47 62542
48 62630
49 62752
50 62736
51 62737
52 62749
53 62754
54 62824
55 62827
56 62845
57 62855
58 62857
59 62963
60 63035
61 63006
62 63011
63 63012
64 63015
65 63043
66 63072
67 63111
68 63128
69 63149
70 63169
71 63176
72 63193
73 63216
74 63273
75 63277
76 63358
77 63371
78 63428
79 63457
80 63467
81 63468
82 63478
83 63494
84 63553
85 63528
86 63563
87 63573
88 63578
89 63579
90 63591
91 63606
92 63630
93 63632
94 63640
95 63755
96 63812
97 63824
98 63849
99 63852
100 63853
101 63865
102 63875
103 63894
104 63898
105 63899
106 63900
107 63909
108 63935
109 63957
110 63958


In [ ]:
connection.close()
#terminate connections and drop the db engine
engine.dispose()